In [150]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler

In [151]:
fashion = fetch_openml("Fashion-MNIST", as_frame=False)

/Users/sml/anaconda3/envs/Torch_PY38/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [152]:
class LogicLinear(nn.Module):
    def __init__(self,in_, out_):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_,256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.ReLU(),
            nn.Linear(64,out_)
        )
    def forward(self,x):
        x = self.layers(x)
        return x

In [153]:
data = fashion.data
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [154]:
target = fashion.target
target

array(['9', '0', '0', ..., '8', '1', '5'], dtype=object)

In [155]:
data.shape, target.shape

((70000, 784), (70000,))

In [156]:
type(data), type(target)
data = pd.DataFrame(data)
target = pd.DataFrame(target)

In [157]:
total = pd.concat([data,target], axis=1)
total1 = total.sample(n=30000)

In [158]:
total1

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,0
55537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
54337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,205.0,203.0,198.0,199.0,102.0,0.0,0.0,0.0,0.0,8
23831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
59194,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,0.0,14.0,67.0,19.0,0.0,0.0,0.0,0.0,2
41450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,237.0,199.0,...,165.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
48229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
25744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [159]:
data = total1.iloc[:,:-1]
target = total1.iloc[:,-1]
data = np.array(data)
target = np.array(target)


In [160]:
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [161]:
type(target), type(target[:])

(numpy.ndarray, numpy.ndarray)

In [162]:
target.shape
target = target.reshape(-1,1)

In [163]:
from sklearn.preprocessing import OneHotEncoder

target = OneHotEncoder().fit_transform(target).toarray()
target.shape

(30000, 10)

In [164]:
target

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

